# Silver Transformation

In [1]:
from pyspark.sql.functions import col,to_date,when
from delta import DeltaTable

StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 3, Finished, Available, Finished)

In [15]:
processing_date = "2026-01-01"
workspace="fabric_PROD"

StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 17, Finished, Available, Finished)

In [16]:
# fabric config
fabric_bronze_path=f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/lms_LH_Bronze.Lakehouse/Tables/student_table"
fabric_silver_path=f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/lms_LH_Silver.Lakehouse/Tables/student_table"

StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 18, Finished, Available, Finished)

## Data Transformation

In [4]:
# read today data
df = (
    spark.read.format("delta").load(fabric_bronze_path)
    .filter(col("Processing_Date")==str(processing_date))
)

StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 6, Finished, Available, Finished)

### Drop duplicates & handle null values

In [5]:
#drop duplicates
print("Total Records Before Removing Duplicates:",df.count())
df_nodup = df.dropDuplicates()
print("Total Records After Removing Duplicates:",df_nodup.count())
# handle null values
# drop rows with missing values
print('Count before dropping missing criticial data rows : ', df_nodup.count() )
df_dropped = df_nodup.dropna(subset=['Student_ID','Course_ID','Enrollment_Date'])
print('Count After dropping missing criticial data rows : ', df_dropped.count() )
# fill rows with null values
df_filled = df_dropped.fillna({
                "Age": 0,
                "Gender": "Unknown",
                "Status": "In-progress",
                "Final_Grade": "N/A",
                "Attendance_Rate": 0.0,
                "Time_Spent_on_Course_hrs": 0.0,
                "Assignments_Completed": 0,
                "Quizzes_Completed": 0,
                "Forum_Posts": 0,
                "Messages_Sent": 0,
                "Quiz_Average_Score": 0.0,
                "Assignment_Average_Score": 0.0,
                "Project_Score": 0.0,
                "Extra_Credit": 0.0,
                "Overall_Performance": 0.0,
                "Feedback_Score": 0.0,
                "Parent_Involvement": "Unknown",
                "Demographic_Group": "Unknown",
                "Internet_Access": "Unknown",
                "Learning_Disabilities": "Unknown",
                "Preferred_Learning_Style": "Unknown",
                "Language_Proficiency": "Unknown",
                "Participation_Rate": "Unknown",
                "Completion_Time_Days": 0,
                "Performance_Score": 0.0,
                "Course_Completion_Rate": 0.0,
                "Completion_Date": '12/31/9999'

            })
# 

StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 7, Finished, Available, Finished)

Total Records Before Removing Duplicates: 26
Total Records After Removing Duplicates: 26
Count before dropping missing criticial data rows :  26
Count After dropping missing criticial data rows :  26


### Standardize Date Format

In [6]:
# date format -> month day year
df_format = (
    df_filled.withColumn("Enrollment_Date", to_date(col("Enrollment_Date"), "M/d/yyyy"))
         .withColumn("Completion_Date",to_date(col('Completion_Date'), "M/d/yyyy"))
)


StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 8, Finished, Available, Finished)

### Check for logical consistency

In [7]:
# check if completion date > enrollment date
df_consistent_check = df_format.filter(col("Completion_Date") >= col("Enrollment_Date")) 

StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 9, Finished, Available, Finished)

## Business Centric Transformation

### 

### Performance Score & Course Completion Rate

In [8]:
# We are subtracting CompletionDate - Enrollment_Date
# We are converting that to integer
df_days = df_consistent_check.withColumn("Completion_Time_Days", (col("Completion_Date") - col("Enrollment_Date")  ).cast('int'))
#Logic : (Quiz_Average_Score * 0.2) + (Assignment_Average_Score * 0.2) + (Project_Score * 0.1)
df_score = df_days.withColumn("Performance_Score", ( 
                                                    (col('Quiz_Average_Score') * 0.2 ) + 
                                                    (col('Assignment_Average_Score')* 0.2) +
                                                    (col('Project_Score')* 0.1) \
                                                )

)
# course completion rate for the enrolled project
df_completion = (
    df_score.withColumn("Course_Completion_Rate", when( col("Completion_Time_Days") <=90 , "On-Time").otherwise('Delayed'))
)


StatementMeta(, e72e869d-2aa3-422b-8424-4ed4bbef3349, 10, Finished, Available, Finished)

## Writing To Silver Table

### Use Delta lake upsert

In [ ]:
# creating temperory table for current date data
# df_completion.createOrReplaceTempView("student_table_temp")
# check if delta table exists in fabric silver path
if not DeltaTable.isDeltaTable(spark,fabric_silver_path):
    print("Table does not exists, creating new table:student_table")
    df_completion.write.format("delta").mode("overwrite").save(fabric_silver_path)
else:
    print("Table exists, updating table")
    delta_tbl = DeltaTable.forPath(spark,fabric_silver_path)
    (
        delta_tbl.alias("t").merge(
            df_completion.alias("s"),
            "t.Student_ID = s.Student_ID AND t.Course_ID = s.Course_ID"
        ).whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    # get metrics
    history_df = delta_tbl.history(1)  # Get the latest operation history
    # Extract metrics from the history DataFrame
    operation_metrics = history_df.select("operationMetrics").collect()[0][0]

    # Extract specific metrics
    rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
    rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
    rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
    rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted) 

    print('Total rows of table: ',delta_tbl.toDF().count())
    print("Merge Metrics:")
    print(f"Rows inserted: {rows_inserted}")
    print(f"Rows updated: {rows_updated}")
    print(f"Rows deleted: {rows_deleted}")
    print(f"Total rows affected: {rows_affected}")